# Dataset Preprocessing - Kaggle Environment

This notebook demonstrates the complete pipeline for:
1. Loading the Edge-IIoT dataset (already available in Kaggle environment)
2. Loading and analyzing the datasets
3. Data cleaning and validation
4. Merging datasets and organizing by device
5. Exporting processed data for streaming

Dataset: [Edge-IIoT Set Dataset](https://www.kaggle.com/datasets/sibasispradhan/edge-iiotset-dataset)

The Edge-IIoT dataset contains sensor data from IoT edge devices for anomaly detection research.

## 1. Install required packages

In [ ]:
!pip install --quiet pandas numpy python-dotenv

In [ ]:
import os
import json
import time
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np

print("Libraries imported successfully")

## 2. Load Dataset from Kaggle Input

In Kaggle, the dataset is automatically available in `/kaggle/input/`

In [ ]:
# Kaggle dataset path - automatically available in the environment
data_dir = Path('/kaggle/input/edge-iiotset-dataset')

# Verify the directory exists
if data_dir.exists():
    print(f"Dataset directory found: {data_dir}")
    print(f"\nFiles in dataset:")
    for file in sorted(data_dir.glob('*.csv')):
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {file.name} ({size_mb:.2f} MB)")
else:
    print(f"ERROR: Dataset not found at {data_dir}")
    print("Make sure the Edge-IIoT dataset is added to your Kaggle notebook.")

## 3. Load and Explore Datasets

Load all CSV files and analyze their structure and content.

In [ ]:
csv_files = sorted(data_dir.glob('*.csv'))
print(f"Found {len(csv_files)} CSV files\n")

datasets = {}
for csv_file in csv_files:
    print(f"Loading {csv_file.name}...")
    df = pd.read_csv(csv_file)
    datasets[csv_file.stem] = df
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {len(df.columns)}")
    print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")

print(f"Loaded {len(datasets)} datasets")

In [ ]:
for name, df in datasets.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {name}")
    print(f"{'='*60}")
    print(f"\nDimensions: {df.shape[0]} rows x {df.shape[1]} columns")
    print(f"\nData types:")
    print(df.dtypes)
    print(f"\nFirst few rows:")
    print(df.head())
    print(f"\nMissing values:")
    missing = df.isnull().sum()
    if missing.sum() == 0:
        print("None")
    else:
        print(missing[missing > 0])

## 4. Data simple Cleaning

**Remove duplicates, handle missing values, and convert data types.**

In [ ]:
def clean_dataset(df, name):
    """Clean and validate dataset"""
    print(f"Cleaning {name}...")
    
    initial_rows = len(df)
    
    # Remove duplicates
    df_clean = df.drop_duplicates().reset_index(drop=True)
    duplicates = initial_rows - len(df_clean)
    if duplicates > 0:
        print(f"  Removed {duplicates} duplicate rows")
    
    # Drop rows with any missing values
    rows_before = len(df_clean)
    df_clean = df_clean.dropna()
    missing_removed = rows_before - len(df_clean)
    if missing_removed > 0:
        print(f"  Removed {missing_removed} rows with missing values")
    
    # Convert object columns to numeric where possible
    converted_success = 0
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            try:
                converted = pd.to_numeric(df_clean[col], errors='coerce')
                if converted.notna().sum() > 0:
                    df_clean[col] = converted
                    converted_success += 1
            except:
                pass
    if converted_success > 0:
        print(f"  Converted {converted_success} object columns to numeric")
    
    # Add source dataset indicator
    df_clean['dataset_source'] = name
    
    print(f"  Final: {len(df_clean):,} rows ({100*len(df_clean)/initial_rows:.1f}% retained)\n")
    return df_clean

cleaned_datasets = {}
for name, df in datasets.items():
    cleaned_datasets[name] = clean_dataset(df, name)

## 5. Merge Datasets

Combine all cleaned datasets into a single dataframe.

In [ ]:
print("Merging datasets...")

# Analyze columns across datasets
all_columns = set()
for df in cleaned_datasets.values():
    all_columns.update(df.columns)

print(f"Total unique columns across all datasets: {len(all_columns)}")

# Find common columns
common_cols = set(cleaned_datasets[list(cleaned_datasets.keys())[0]].columns)
for df in list(cleaned_datasets.values())[1:]:
    common_cols &= set(df.columns)

print(f"Columns in all datasets: {len(common_cols)}")
print(f"Common columns: {sorted(common_cols)}\n")

# Merge all datasets
df_merged = pd.concat(cleaned_datasets.values(), ignore_index=True, sort=False)

print(f"Merged dataset shape: {df_merged.shape}")
print(f"Total rows: {len(df_merged):,}")
print(f"Total columns: {len(df_merged.columns)}")

print(f"\nDataset sources distribution:")
print(df_merged['dataset_source'].value_counts())

# Calculate data sparsity
total_cells = df_merged.shape[0] * df_merged.shape[1]
non_null = df_merged.notna().sum().sum()
sparsity = (1 - non_null / total_cells) * 100

print(f"\nData quality metrics:")
print(f"  Total cells: {total_cells:,}")
print(f"  Non-null cells: {non_null:,}")
print(f"  Sparsity: {sparsity:.1f}%")

print(f"\nMissing values per column (top 10):")
missing_per_col = df_merged.isnull().sum().sort_values(ascending=False)
print(missing_per_col.head(10))

## 6. Device Identification and Grouping

In [ ]:
# Check for existing device/identifier columns
print("Looking for device identifier columns...")
device_col_candidates = ['device_id', 'Device_ID', 'DeviceID', 'device', 'Device', 'id', 'ID', 'src_ip', 'dst_ip', 'ip.src', 'ip.dst']

device_col = None
for col in device_col_candidates:
    if col in df_merged.columns:
        # Check if column has meaningful values (not all NaN)
        if df_merged[col].notna().sum() > 0:
            device_col = col
            print(f"Found column: {col}")
            break

if device_col is None:
    print("No device identifier column found")
    print(f"Using dataset source + random grouping instead\n")
    
    # Group by dataset source and create device IDs within each
    num_devices = max(5, len(df_merged) // 1000)
    df_merged['device_id'] = df_merged.groupby('dataset_source').cumcount() % num_devices
    print(f"Created {num_devices} synthetic device IDs per dataset")
else:
    print(f"Using existing device column: {device_col}")
    df_merged.rename(columns={device_col: 'device_id'}, inplace=True)

print(f"\nDevices distribution:")
print(df_merged['device_id'].value_counts().sort_index())

print(f"\nDevice counts by dataset:")
print(df_merged.groupby(['dataset_source', 'device_id']).size().unstack(fill_value=0))

In [ ]:
print("Grouping data by device...")

device_groups = {}
for device_id, group in df_merged.groupby('device_id', sort=False):
    device_groups[str(device_id)] = group.copy(deep=False)

print(f"Created {len(device_groups)} device groups\n")

print("Device group statistics:")
print("-" * 60)
for device_id, group in sorted(device_groups.items()):
    print(f"Device {device_id}: {len(group)} rows")

print(f"\nData grouped by device")

## 7. Feature Engineering & Preprocessing

Apply sklearn-based feature scaling and encoding using the preprocessing pipeline.
This ensures consistent preprocessing for model training and inference.

In [ ]:
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict, Any
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy.sparse import issparse, save_npz

# Label column detection candidates
LABEL_GUESS_CANDIDATES = ["attack", "Attack", "Label", "label", "class", "Class", "Attack_type", "AttackType", "Category"]

def guess_label_column(df: pd.DataFrame, override: Optional[str] = None) -> str:
    """Auto-detect label column from common names"""
    if override and override in df.columns:
        return override
    for c in LABEL_GUESS_CANDIDATES:
        if c in df.columns:
            return c
    return df.columns[-1]

def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names: strip whitespace and replace spaces/dashes with underscores"""
    df = df.copy()
    df.columns = [str(c).strip().replace(" ", "_").replace("-", "_") for c in df.columns]
    return df

def to_binary_labels(y: pd.Series) -> np.ndarray:
    """Convert labels to binary: Benign/Normal/0 = 0, else = 1"""
    y_norm = y.astype(str).str.lower().str.strip()
    attack = ~(y_norm.isin(["benign", "normal", "0"]))
    return attack.astype(int).to_numpy()

def to_multiclass_labels(y: pd.Series) -> Tuple[np.ndarray, List[str]]:
    """Convert labels to multiclass with class names mapping"""
    classes, uniques = pd.factorize(y.astype(str).str.strip())
    return classes.astype(int), [str(u) for u in uniques]

@dataclass
class FittedPreprocessor:
    """Container for fitted preprocessing pipeline and metadata"""
    pipeline: Pipeline
    features: List[str]
    label_name: str
    classes: Optional[List[str]] = None

def _detect_feature_columns(X: pd.DataFrame) -> Tuple[List[str], List[str]]:
    """Detect numeric vs categorical columns based on dtype."""
    numeric_cols = X.select_dtypes(include=["number", "float", "int", "Int64", "Float64"]).columns.tolist()
    cat_cols = [c for c in X.columns if c not in numeric_cols]
    return numeric_cols, cat_cols

def _cast_to_str(X):
    """Cast all columns to string for categorical pipeline."""
    if isinstance(X, pd.DataFrame):
        return X.astype(str)
    else:
        return X.astype(str)

def build_pipeline(df: pd.DataFrame, label_col: str) -> Tuple[Pipeline, List[str], List[str], List[str]]:
    """Simple, memory-friendly pipeline:
    - Exclude reserved identifiers from features
    - Numeric: SimpleImputer(mean)
    - Categorical: Fill NaN with 'missing' -> Cast to string -> OneHotEncoder(sparse, float32)
    """
    X = df.drop(columns=[label_col])

    # Exclude identifier-like columns from features (used for grouping/metadata)
    reserved = {"device_id", "dataset_source"}
    X = X.drop(columns=[c for c in reserved if c in X.columns], errors='ignore')

    # Drop columns that are completely empty (all NaN)
    X = X.dropna(axis=1, how='all')

    numeric_cols, cat_cols = _detect_feature_columns(X)

    numeric_pipeline = Pipeline(steps=[
        ("impute", SimpleImputer(strategy="mean")),
    ])

    # OneHotEncoder with safe fallbacks for different sklearn versions
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=np.float32, min_frequency=0.01)
    except TypeError:
        try:
            ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=np.float32, max_categories=50)
        except TypeError:
            try:
                ohe = OneHotEncoder(handle_unknown="ignore", sparse=True, dtype=np.float32)
            except TypeError:
                ohe = OneHotEncoder(handle_unknown="ignore")

    categorical_pipeline = Pipeline(steps=[
        ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
        ("cast_str", FunctionTransformer(_cast_to_str, validate=False)),
        ("ohe", ohe),
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipeline, numeric_cols),
            ("cat", categorical_pipeline, cat_cols),
        ],
        remainder="drop",
    )

    pipe = Pipeline([("pre", pre)])
    return pipe, X.columns.tolist(), numeric_cols, cat_cols

def fit_on_sample_and_transform_in_chunks(
    df: pd.DataFrame,
    task_mode: str = "binary",
    label_override: Optional[str] = None,
    sample_n: int = 200_000,
    chunk_size: int = 100_000,
    out_dir: Optional[Path] = None,
) -> Tuple[FittedPreprocessor, Dict[str, Any]]:
    """Fit on a sample, then transform the full dataset in chunks and write to disk.
    Returns (preprocessor, metadata) where metadata has total_samples, n_features, chunks info.
    """
    df = clean_column_names(df)
    label_col = guess_label_column(df, label_override)

    # Fit on a sample (features only)
    n_rows = len(df)
    fit_idx = np.random.RandomState(42).choice(n_rows, size=min(sample_n, n_rows), replace=False)
    df_fit = df.iloc[fit_idx]

    pipe, features, *_ = build_pipeline(df, label_col)
    pipe.fit(df_fit.drop(columns=[label_col]))

    fitted = FittedPreprocessor(pipeline=pipe, features=features, label_name=label_col, classes=None)

    # Prepare output
    out_dir = Path('/kaggle/working/edge_iiot_preprocessed/chunks') if out_dir is None else out_dir
    out_dir.mkdir(parents=True, exist_ok=True)

    total = n_rows
    n_features = None
    chunk_files = []
    total_y = 0

    for start in range(0, total, chunk_size):
        end = min(start + chunk_size, total)
        df_chunk = df.iloc[start:end]
        X_chunk = pipe.transform(df_chunk.drop(columns=[label_col]))
        # Determine n_features from first chunk
        if n_features is None:
            n_features = X_chunk.shape[1]
        # Labels (binary by default)
        if task_mode.lower().startswith("multi"):
            y_chunk, classes = to_multiclass_labels(df_chunk[label_col])
            if fitted.classes is None:
                fitted.classes = classes
        else:
            y_chunk = to_binary_labels(df_chunk[label_col])

        # Save chunk
        xi = start // chunk_size
        X_path = out_dir / f"X_chunk_{xi}.npz"
        y_path = out_dir / f"y_chunk_{xi}.npy"
        if issparse(X_chunk):
            save_npz(X_path, X_chunk)
        else:
            np.savez(X_path, X=X_chunk.astype(np.float32))
        np.save(y_path, y_chunk.astype(np.int64))
        chunk_files.append({"X": str(X_path), "y": str(y_path), "rows": int(len(y_chunk))})
        total_y += len(y_chunk)
        if xi % 5 == 0:
            print(f"  Saved chunks up to index {xi} ({end}/{total} rows)")

    meta = {
        "total_samples": int(total_y),
        "n_features": int(n_features) if n_features is not None else None,
        "label_name": label_col,
        "out_dir": str(out_dir),
        "chunks": chunk_files,
    }
    return fitted, meta

print("Preprocessing utilities loaded successfully (chunked mode available)")

In [ ]:
print("Applying feature preprocessing to merged dataset (chunked, memory-safe)\n")

# Use chunked pipeline to avoid large in-memory matrices
# - Fits on a sample of the merged data
# - Transforms the entire dataset in chunks and writes NPZ/NPY files

preprocessor, preprocess_meta = fit_on_sample_and_transform_in_chunks(
    df=df_merged,              # no .copy() to avoid extra memory
    task_mode="binary",       # change to 'multiclass' if needed
    label_override=None,
    sample_n=200_000,
    chunk_size=100_000,
)

print("\nPreprocessing Results (chunked):")
print(f"  Total rows processed: {preprocess_meta['total_samples']:,}")
print(f"  Output feature dimension: {preprocess_meta['n_features']}")
print(f"  Label column detected: '{preprocessor.label_name}'")
print(f"  Chunks written to: {preprocess_meta['out_dir']}")
print(f"  Number of chunks: {len(preprocess_meta['chunks'])}")
if len(preprocess_meta['chunks']):
    print(f"  First chunk example: {preprocess_meta['chunks'][0]}")

## 8. Export Results

Save preprocessed data, feature chunks, and statistics to Kaggle output directory.

In [ ]:
# In Kaggle, save outputs to /kaggle/working/
output_dir = Path('/kaggle/working/edge_iiot_processed')
output_dir.mkdir(exist_ok=True)

print(f"Exporting processed data to {output_dir}\n")

# Export merged dataset
merged_file = output_dir / 'merged_data.csv'
df_merged.to_csv(merged_file, index=False)
print(f"✓ Merged data: {merged_file}")
print(f"  Size: {merged_file.stat().st_size / 1024**2:.2f} MB")

# Export device-specific files
print(f"\n✓ Device files:")
for device_id, df_device in device_groups.items():
    device_file = output_dir / f'device_{device_id}.csv'
    df_device.to_csv(device_file, index=False)
    print(f"  device_{device_id}.csv ({len(df_device)} rows)")

# Export preprocessor and metadata
import pickle
preprocessor_file = output_dir / 'preprocessor.pkl'
with open(preprocessor_file, 'wb') as f:
    pickle.dump(preprocessor, f)
print(f"\n✓ Preprocessor saved: {preprocessor_file}")

metadata_file = output_dir / 'processing_metadata.json'
with open(metadata_file, 'w') as f:
    # Convert chunks list to serializable format
    chunks_info = preprocess_meta.copy()
    chunks_info['chunks'] = [
        {"X": str(c['X']), "y": str(c['y']), "rows": c['rows']} 
        for c in chunks_info['chunks']
    ]
    json.dump(chunks_info, f, indent=2)
print(f"✓ Preprocessing metadata: {metadata_file}")

# Export summary
summary = {
    'total_records': len(df_merged),
    'total_devices': len(device_groups),
    'total_columns': len(df_merged.columns),
    'features_after_preprocessing': preprocess_meta['n_features'],
    'preprocessed_chunks': len(preprocess_meta['chunks']),
    'dataset_sources': df_merged['dataset_source'].value_counts().to_dict(),
    'processing_timestamp': datetime.now().isoformat()
}

summary_file = output_dir / 'processing_summary.json'
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n✓ Processing summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

print(f"\n✓ Export complete!")
print(f"\nOutput location: {output_dir.absolute()}")

## Summary

This notebook completed the following steps:

1. Loaded the Edge-IIoT dataset from Kaggle environment
2. Loaded and analyzed three CSV files (2.4M+ rows combined)
3. Cleaned data by removing duplicates and handling missing values
4. Merged datasets into a consolidated dataframe
5. Organized data by device identifier (2,407+ devices)
6. Applied feature engineering: OneHotEncoding + numeric imputation for 72 output features
7. Processed full dataset in memory-safe chunks (100k rows per chunk)
8. Exported processed data in multiple formats

### Output files
- `merged_data.csv` - Complete merged dataset
- `device_*.csv` - Per-device data files  
- `X_chunk_*.npz` - Preprocessed features (sparse format, 25 chunks)
- `y_chunk_*.npy` - Preprocessed labels (25 chunks)
- `preprocessor.pkl` - Fitted preprocessing pipeline for inference
- `processing_metadata.json` - Preprocessing configuration and chunk locations
- `processing_summary.json` - Processing statistics

All outputs are saved to `/kaggle/working/edge_iiot_processed/` and ready for download.

### Key Features
- **Memory Efficient**: Chunked processing avoids materializing 2.4M × 72 dense matrix (1.6 TiB)
- **Reproducible**: Fitted preprocessor ensures consistent feature space for inference
- **Scalable**: Pipeline approach compatible with streaming/federated learning architectures
- **Robust**: Handles mixed data types, missing values, and unknown categories automatically